In [1]:
import numpy as np
import os
from utils.utilities import *
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
import tensorflow as tf

## Prepare data

In [3]:
har_x_train, har_y_train, list_ch_train = read_data(data_path="./dataset/UCI HAR Dataset/", split="train") # train
har_x_test, har_y_test, list_ch_test = read_data(data_path="./dataset/UCI HAR Dataset/", split="train") # test

In [4]:
X_train, X_valid, y_train, y_valid = train_test_split(har_x_train, har_y_train, stratify=har_y_train, random_state=42)

In [5]:
X_train.shape, y_train.shape

((5514, 128, 9), (5514,))

# Build DNN model

## Hyperparameters

In [6]:
batch_size = 60
seq_len = 128
learning_rate = 0.01
n_epochs = 100

n_classes = 7 # from 1 - 6
n_channels = 9

reshape all data into 1D

In [7]:
X_train.shape, y_train.shape

((5514, 128, 9), (5514,))

In [8]:
X_train = X_train.reshape(-1, seq_len * n_channels)
X_valid = X_valid.reshape(-1, seq_len * n_channels)
har_x_test = har_x_test.reshape(-1, seq_len * n_channels)

In [9]:
X_train.shape, y_train.shape

((5514, 1152), (5514,))

## the model

In [10]:
tf.reset_default_graph()

X = tf.placeholder(tf.float32, (None, seq_len * n_channels), name="X")
y = tf.placeholder(tf.int32, (None), name="y")


In [11]:
with tf.name_scope("dnn"):
    
    dense1 = tf.layers.dense(X, 300, activation=tf.nn.relu, name="dense1")
    dense2 = tf.layers.dense(dense1, 100, activation=tf.nn.relu, name="dense2")
    dense3 = tf.layers.dense(dense2, 50, activation=tf.nn.relu, name="dense3")

    logits = tf.layers.dense(dense3, n_classes)    

In [13]:
with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=y)
    loss = tf.reduce_mean(xentropy, name="loss")

with tf.name_scope("train"):
    training_op = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss)

with tf.name_scope("eval"):
    #correct = tf.equal(tf.argmax(logits, 1), tf.argmax(y, 1))
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32), name="accuracy")

In [16]:
init = tf.global_variables_initializer()
saver = tf.train.Saver()

with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epochs):
        for X_batch, y_batch in get_batches(X_train, y_train, batch_size):
            train_acc =  sess.run([training_op], feed_dict={X: X_batch, y: y_batch})
        valid_acc = accuracy.eval(feed_dict={X: X_valid, y: y_valid})
        
        print("Epoch: ", epoch, "Valid accuracy: ", valid_acc)
    saver.save(sess, "./model/har_cnn_model.ckpt")

In [18]:
with tf.Session() as sess:
    saver.restore(sess, "./model/har_cnn_model.ckpt")
    test_acc = accuracy.eval(feed_dict={X: har_x_test, y: har_y_test})
    print("Test Accuracy: ", test_acc)

INFO:tensorflow:Restoring parameters from ./model/har_cnn_model.ckpt
Test Accuracy:  0.960827


## Add Batch Norm to the model

In tensorflow we use `tf.layers.batch_normalization` for batch normalization
  - first apply the function to each dense layer, including logits layer
  - second need training the `tf.GraphKeys.UPDATE_OPS` after optimizer

fine tuning the model:
  1. training much slowly after batch norm added
  2. use `elu` to replace `relu`, but slow and worse accuracy
  3. use `batch_size` 600, but got worse accuracy

In [104]:
batch_size = 60

In [105]:
tf.reset_default_graph()

X = tf.placeholder(tf.float32, (None, seq_len * n_channels), name="X")
y = tf.placeholder(tf.int32, (None), name="y")
training = tf.placeholder_with_default(False, shape=(), name="training")

In [106]:
with tf.name_scope("dnn"):
    
    dense1 = tf.layers.dense(X, 300, name="dense1")
    bn1 = tf.layers.batch_normalization(dense1, training=training)
    bn1_act = tf.nn.relu(bn1)
    
    dense2 = tf.layers.dense(bn1_act, 100, name="dense2")
    bn2 = tf.layers.batch_normalization(dense2, training=training)
    bn2_act = tf.nn.relu(bn2)
    
    dense3 = tf.layers.dense(bn2_act, 50, name="dense3")
    bn3 = tf.layers.batch_normalization(dense3, training=training)
    bn3_act = tf.nn.relu(bn3)

    logits_before_bn = tf.layers.dense(bn3_act, n_classes)
    logits = tf.layers.batch_normalization(logits_before_bn, training=training)


In [107]:
with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=y)
    loss = tf.reduce_mean(xentropy, name="loss")

with tf.name_scope("train"):   
    extra_update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
    with tf.control_dependencies(extra_update_ops):
        training_op = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss) 

with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32), name="accuracy")

In [108]:
init = tf.global_variables_initializer()
saver = tf.train.Saver()

with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epochs):
        for X_batch, y_batch in get_batches(X_train, y_train, batch_size):
            train_acc =  sess.run([training_op], feed_dict={X: X_batch, y: y_batch, training:True})
        valid_acc = accuracy.eval(feed_dict={X: X_valid, y: y_valid})
        
        if epoch %10 == 0:
            print("Epoch: ", epoch, "Valid accuracy: ", valid_acc)
    saver.save(sess, "./model/har_cnn_model.ckpt")

Epoch:  0 Valid accuracy:  0.407508
Epoch:  10 Valid accuracy:  0.782916
Epoch:  20 Valid accuracy:  0.931991
Epoch:  30 Valid accuracy:  0.94124
Epoch:  40 Valid accuracy:  0.933079
Epoch:  50 Valid accuracy:  0.932535
Epoch:  60 Valid accuracy:  0.928727
Epoch:  70 Valid accuracy:  0.914037
Epoch:  80 Valid accuracy:  0.931447
Epoch:  90 Valid accuracy:  0.917845


In [109]:
with tf.Session() as sess:
    saver.restore(sess, "./model/har_cnn_model.ckpt")
    test_acc = accuracy.eval(feed_dict={X: har_x_test, y: har_y_test})
    print("Test Accuracy: ", test_acc)

INFO:tensorflow:Restoring parameters from ./model/har_cnn_model.ckpt
Test Accuracy:  0.959059
